In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tfrecorder import TFrecorder
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%pylab inline

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Populating the interactive namespace from numpy and matplotlib


# input_fn's

In [3]:
tfr = TFrecorder()
def input_fn_maker(path, data_info_path, shuffle=False, batch_size = 1, epoch = 1, padding = None):
    def input_fn():
        filenames = tfr.get_filenames(path=path, shuffle=shuffle)
        dataset=tfr.get_dataset(paths=filenames, data_info=data_info_path, shuffle = shuffle, 
                            batch_size = batch_size, epoch = epoch, padding =padding)
        iterator = dataset.make_one_shot_iterator()
        return iterator.get_next()
    return input_fn

padding_info = ({'image':[784,],'label':[]})
test_input_fn = input_fn_maker('mnist_tfrecord/test/',  'mnist_tfrecord/data_info.csv',batch_size = 512,
                               padding = padding_info)
train_input_fn = input_fn_maker('mnist_tfrecord/train/',  'mnist_tfrecord/data_info.csv', shuffle=True, batch_size = 128,
                               padding = padding_info)
train_eval_fn = input_fn_maker('mnist_tfrecord/train/',  'mnist_tfrecord/data_info.csv', batch_size = 512,
                               padding = padding_info)

In [25]:
def model_fn(features, mode):
    #Hparams
    relation_activation = tf.nn.sigmoid
    # inputs
    # reshape 784维的图片到28x28的平面表达，1为channel数
    
    old_image = tf.reshape(features['image'],[-1,28,28,1])
    new_image = old_image
    # labels
    old_label = features['label']
    new_label = features['label']
    
    #old_image = tf.reshape(features['old_image'],[-1,28,28,1])
    #new_image = tf.reshape(features['new_image'],[-1,28,28,1])
    # labels
    #old_label = features['old_label']
    #new_label = features['new_label']
    
    # shape: [None,28,28,1]
    old_conv1 = tf.layers.conv2d(
        inputs=old_image,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu,
        name = 'conv1',reuse=tf.AUTO_REUSE)# share weight for old_image and new_image
    
    new_conv1 = tf.layers.conv2d(
        inputs=new_image,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu,
        name = 'conv1',reuse=tf.AUTO_REUSE)
    
    # shape: [None,28,28,32]
    old_pool1 = tf.layers.max_pooling2d(inputs=old_conv1, pool_size=[2, 2], strides=2)
    new_pool1 = tf.layers.max_pooling2d(inputs=new_conv1, pool_size=[2, 2], strides=2)
    # shape: [None,14,14,32]
    
    relation1_input = tf.layers.flatten(tf.concat(values=[old_pool1, new_pool1], axis=-1))
    
    relation1 = tf.layers.dense(inputs=relation1_input, units=1, activation=relation_activation, name= 'relation1')
    
    
    old_conv2 = tf.layers.conv2d(
        inputs= old_pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu,
        name = 'conv2',reuse=tf.AUTO_REUSE)
    
    new_conv2 = tf.layers.conv2d(
        inputs= new_pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu,
        name = 'conv2',reuse=tf.AUTO_REUSE)
    
    # shape: [None,14,14,64]
    old_pool2 = tf.layers.max_pooling2d(inputs=old_conv2, pool_size=[2, 2], strides=2)
    new_pool2 = tf.layers.max_pooling2d(inputs=new_conv2, pool_size=[2, 2], strides=2)
    
    # shape: [None,7,7,64]
    old_pool2_flat = tf.reshape(old_pool2, [-1, 7 * 7 * 64],name='re1')
    new_pool2_flat = tf.reshape(new_pool2, [-1, 7 * 7 * 64],name='re2')

    relation2_input = tf.concat(values=[old_pool2_flat, new_pool2_flat], axis=-1)
    relation2 = tf.layers.dense(inputs=relation2_input, units=1, activation=relation_activation, name= 'relation2')
    
    # shape: [None,3136]
    old_dense1 = tf.layers.dense(inputs=old_pool2_flat, units=1024, activation=tf.nn.relu, name= 'dense1',reuse=tf.AUTO_REUSE)
    new_dense1 = tf.layers.dense(inputs=new_pool2_flat, units=1024, activation=tf.nn.relu, name= 'dense1',reuse=tf.AUTO_REUSE)
    
    # shape: [None,1024]
    old_logits = tf.layers.dense(inputs=old_dense1, units=10, name= 'output',reuse=tf.AUTO_REUSE)
    new_logits = tf.layers.dense(inputs=new_dense1, units=10, name= 'output',reuse=tf.AUTO_REUSE)
    
    # shape: [None,10]
    
    predictions = {
        "old_image":old_image,
        "new_image":new_image,
        "old_label": old_label,
        "new_label": new_label,
        "relation1":relation1,
        "relation2":relation2,
        "old_class": tf.argmax(input=old_logits, axis=1),
        "new_class": tf.argmax(input=new_logits, axis=1),
        "old_probability": tf.nn.softmax(old_logits, name="old_softmax_tensor"),
        "new_probability": tf.nn.softmax(new_logits, name="new_softmax_tensor")
        }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    old_loss = tf.losses.sparse_softmax_cross_entropy(labels=old_label, logits=old_logits)
    new_loss = tf.losses.sparse_softmax_cross_entropy(labels=new_label, logits=new_logits)
    
    relation = tf.reduce_mean(relation1+relation2,axis=0)
    
    loss = old_loss*relation + new_loss*(1-relation)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    eval_metric_ops = {"old_accuracy": tf.metrics.accuracy(labels=old_label, predictions=predictions["old_class"]),
                       "new_accuracy": tf.metrics.accuracy(labels=new_label, predictions=predictions["new_class"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

# model

In [26]:
mnist_classifier = tf.estimator.Estimator(
    model_fn=model_fn, model_dir="two")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'two', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe1885787f0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# train

In [20]:
mnist_classifier.train(
    input_fn=train_input_fn)

read dataframe from mnist_tfrecord/train/train20000_30000.tfrecord x 6
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from two/model.ckpt-430
INFO:tensorflow:Saving checkpoints for 431 into two/model.ckpt.
INFO:tensorflow:loss = 0.120575, step = 431
INFO:tensorflow:global_step/sec: 5.42848
INFO:tensorflow:loss = 0.0467044, step = 531 (18.423 sec)
INFO:tensorflow:global_step/sec: 5.44896
INFO:tensorflow:loss = 0.0300936, step = 631 (18.352 sec)
INFO:tensorflow:global_step/sec: 5.47131
INFO:tensorflow:loss = 0.019889, step = 731 (18.277 sec)
INFO:tensorflow:global_step/sec: 5.45703
INFO:tensorflow:loss = 0.0146581, step = 831 (18.325 sec)
INFO:tensorflow:Saving checkpoints for 860 into two/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0154384.


# evaluate train set 

In [21]:
eval_results = mnist_classifier.evaluate(input_fn=train_eval_fn)
print('train set')
print(eval_results)

read dataframe from mnist_tfrecord/train/train0_10000.tfrecord x 6
INFO:tensorflow:Starting evaluation at 2018-05-06-07:48:48
INFO:tensorflow:Restoring parameters from two/model.ckpt-860
INFO:tensorflow:Finished evaluation at 2018-05-06-07:49:01
INFO:tensorflow:Saving dict for global step 860: global_step = 860, loss = 0.0281809, new_accuracy = 0.991055, old_accuracy = 0.991055
train set
{'loss': 0.028180944, 'new_accuracy': 0.99105453, 'old_accuracy': 0.99105453, 'global_step': 860}


# evaluate test set 

In [22]:
eval_results = mnist_classifier.evaluate(input_fn=test_input_fn)
print('test set')
print(eval_results)

read dataframe from mnist_tfrecord/test/test0_10000.tfrecord x 1
INFO:tensorflow:Starting evaluation at 2018-05-06-07:49:01
INFO:tensorflow:Restoring parameters from two/model.ckpt-860
INFO:tensorflow:Finished evaluation at 2018-05-06-07:49:04
INFO:tensorflow:Saving dict for global step 860: global_step = 860, loss = 0.0359688, new_accuracy = 0.9886, old_accuracy = 0.9886
test set
{'loss': 0.035968848, 'new_accuracy': 0.98860002, 'old_accuracy': 0.98860002, 'global_step': 860}


# predict

In [27]:
predicts =list(mnist_classifier.predict(input_fn=test_input_fn))

read dataframe from mnist_tfrecord/test/test0_10000.tfrecord x 1
INFO:tensorflow:Restoring parameters from two/model.ckpt-860


# print predictions

In [29]:
predicts[0]['relation1']

array([ 0.54239422], dtype=float32)

In [30]:
predicts[0]['relation2']

array([ 0.58716232], dtype=float32)